In [ ]:
#pip install langchain langchain-community langchain-openai geopandas sqlalchemy geoalchemy2 pymssql PyPDF2 tiktoken==0.5.2 mlflow==2.9.0 chromadb

In [ ]:
# install packages
!pip install langchain
!pip install langchain_openai
!pip install openai
#!pip install unstructured
!pip install chromadb
!pip install pypdf
#!pip install "unstructured[pdf]"

In [ ]:
#pip freeze

In [ ]:
# setup env
import os
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://andy-oa.openai.azure.com/'
os.environ["AZURE_OPENAI_API_KEY"] = '6de7de32e13542e8ac28d05c3f7b51e9'

In [ ]:
# import required packages
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI,VectorDBQA
from langchain.document_loaders import DirectoryLoader
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from langchain.document_loaders import UnstructuredFileLoader
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
# loading text documents
loader = PyPDFLoader(r"C:\Users\WilsonHung\Desktop\11880-retail-important-information-guide.pdf")

#loader = DirectoryLoader('C:\Users\WilsonHung\Desktop', glob='**\*.pdf')
documents = loader.load()

# initialise text splitter
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
# split document
split_docs = text_splitter.split_documents(documents)
# initialise openai embeddings model
from langchain_openai import AzureOpenAIEmbeddings
embeddings = AzureOpenAIEmbeddings(
    azure_deployment="AndyEmbeddingModel",
    openai_api_version="2023-05-15"
)

# store the vectors calculated from the docs into Chroma DB
docsearch = Chroma.from_documents(split_docs, embeddings)

In [ ]:
# Initialise azure openai chatmodel
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
azureopenai_llm = AzureChatOpenAI(deployment_name="AndyChat",
                       max_tokens=1024,
                       openai_api_version="2023-05-15")

In [ ]:
# create the retriever
qa = RetrievalQAWithSourcesChain.from_chain_type(llm = azureopenai_llm,
                                 chain_type="stuff",
                                 retriever=docsearch.as_retriever())
                                 #return_source_documents=True)
# Ask a question
result = qa.invoke("Who is Wilson and what does he specialises?")

In [ ]:
print(result)